In [7]:
!pip install pandas


  Using cached pandas-2.3.3-cp311-cp311-macosx_11_0_arm64.whl.metadata (91 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
Using cached pandas-2.3.3-cp311-cp311-macosx_11_0_arm64.whl (10.8 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pandas]2m2/3 [pandas]


## Testing Methodology & Scenario Design

### Real-World Agent Simulation
**Goal**: Demonstrate evaluation on realistic agent usage patterns

**Agent Profile**: Travel Assistant v1
- Fixed prompt: `artifacts/prompts/prompt_v1.txt`
- Consistent persona: Helpful travel concierge

### Realistic Conversation Scenarios
**Based on actual travel assistant usage**:
- Flight bookings with passenger preferences
- Hotel reservations with amenity requirements
- Multi-step conversations with context
- Error handling and clarification requests
- Price comparisons and recommendations

**User Types**: Business travelers, vacation planners, budget-conscious tourists

### Pattern Embedding Strategy
**Challenge**: Create detectable patterns without obvious artificiality

**Approach**:
1. **Natural Failure Points**: Use realistic scenarios where agents commonly fail
2. **Subtle Issues**: Problems that seem minor but indicate systematic issues
3. **Pattern Density**: 4-5 conversations per pattern for reliable clustering
4. **Mixed Severity**: Range from obvious errors to nuanced preferences

**Embedding Techniques**:
- Date format variations that seem like user input errors
- Partial preference mentions that agents might overlook
- Contextual requirements hidden in conversation flow
- Tool parameter combinations that test validation logic

### Fixed Prompt Design
**Why Fixed**: Control variable to isolate evaluation system performance
- Same prompt across all conversations
- No prompt variations or A/B testing
- Focus on evaluation pipeline, not prompt optimization
- Enables comparison of evaluation results across identical agent behavior

### Evaluation Objectives
- **Pattern Recognition**: Can the system detect embedded failure patterns?
- **Semantic Depth**: Does LLM judge catch issues heuristics miss?
- **Proposal Quality**: Are generated suggestions actionable and relevant?

In [1]:
from src.pipeline.batch_processor import BatchPipelineProcessor

In [2]:

processor = BatchPipelineProcessor()

# Configure the analysis
print("\n⚙️ Configuration:")
print("   Pattern: *.json (single agent version)")
print("   Stages: Intelligent defaults (Heuristic → LLM → Analysis)")
print("   Features: One-by-one progress, staged evaluation, pattern analysis")

# Run the complete batch analysis
result_travel_v1_batch = processor.run_batch_analysis(
    source_pattern="travel_v1_*.json",
    max_conversations=None  # Process all
)



⚙️ Configuration:
   Pattern: *.json (single agent version)
   Stages: Intelligent defaults (Heuristic → LLM → Analysis)
   Features: One-by-one progress, staged evaluation, pattern analysis
🚀 Starting Batch Analysis Pipeline

📁 Stage 1: Loading Conversations
   Looking in directory: /Users/anubhavtomar/Anubhav/repos/agent-eval/data/travel_agent
   Using pattern: travel_v1_*.json
📖 Found 15 files matching 'travel_v1_*.json'
    1/15 ✅ travel_v1_good_001.json   → 1 conversation
    2/15 ✅ travel_v1_issue_007.json  → 1 conversation
    3/15 ✅ travel_v1_issue_011.json  → 1 conversation
    4/15 ✅ travel_v1_issue_010.json  → 1 conversation
    5/15 ✅ travel_v1_issue_006.json  → 1 conversation
    6/15 ✅ travel_v1_issue_001.json  → 1 conversation
    7/15 ✅ travel_v1_issue_003.json  → 1 conversation
    8/15 ✅ travel_v1_issue_002.json  → 1 conversation
    9/15 ✅ travel_v1_issue_009.json  → 1 conversation
   10/15 ✅ travel_v1_good_003.json   → 1 conversation
   11/15 ✅ travel_v1_issue_005.

In [3]:
# result_travel_v1_batch['analysis'][1].__dict__
result_travel_v1_batch['evaluations'][1].__dict__

{'conversation_id': 'travel_v1_issue_007',
 'evaluations': {'heuristic': EvaluatorResult(evaluator_name='heuristic', scores={'format_compliance': 1.0, 'required_fields': 1.0, 'latency_ok': 1.0}, issues=(), confidence=1.0, metadata={'total_turns': 6, 'assistant_turns': 5, 'format_issues': 0, 'field_issues': 0, 'latency_issues': 0, 'latency_ms': 0.012665987014770508}, latency_ms=0.012665987014770508),
  'tool_call': EvaluatorResult(evaluator_name='tool_call', scores={'tool_selection': 1.0, 'param_accuracy': 1.0, 'no_hallucination': 1.0, 'execution_success': 1.0}, issues=(), confidence=0.95, metadata={'total_tool_calls': 2, 'selection_issues': 0, 'param_issues': 0, 'hallucination_issues': 0, 'execution_failures': 0, 'latency_ms': 0.01695798709988594}, latency_ms=0.01695798709988594),
  'tool_causality': EvaluatorResult(evaluator_name='tool_causality', scores={'data_provenance': 0.5}, issues=(Issue(issue_type=<IssueType.TOOL_HALLUCINATION: 'tool_hallucination'>, severity=<IssueSeverity.HIG

In [4]:
# 4. Improvement Proposals
from IPython.display import Markdown


proposals = result_travel_v1_batch["analysis"]
if proposals:
    display(Markdown(f"### 💡 Improvement Proposals ({len(proposals)} generated)"))
    
    for i, proposal in enumerate(proposals, 1):
        display(Markdown(f"""
#### {i}. {proposal.failure_pattern}

**Evidence:** {len(proposal.evidence_ids)} conversations  
**Type:** {proposal.type.value}

**Problem:** {proposal.rationale}

**Solution:** {proposal.proposed_content[:200]}...
---
"""))

### 💡 Improvement Proposals (5 generated)


#### 1. The core technical cause is the assistant's inability to handle missing or incorrectly formatted input parameters and its failure to guide users through error recovery. These interactions lack proactive clarification and fail to solicit necessary user information for successful task completion.

**Evidence:** 15 conversations  
**Type:** tool

**Problem:** The updated schema introduces detailed validation rules for input parameters and provides clear descriptions and examples for users. It ensures that the assistant can better handle missing or malformed inputs by guiding the user with specific error messages and additional guidance on correcting their input.

**Solution:** {
  "required_params": [
    {
      "name": "destination",
      "description": "The airport code or city name for the flight's destination.",
      "type": "string",
      "validation": {
        "n...
---



#### 2. The AI system fails to retrieve or communicate all necessary data from its knowledge base or APIs, leading to partial responses.

**Evidence:** 1 conversations  
**Type:** prompt

**Problem:** The change emphasizes the importance of gathering and verifying comprehensive information before responding. By explicitly instructing the AI to ensure completeness and verify the gathered data, we address the systemic issue of partial responses. This localized modification should prompt the AI to be more diligent in its data retrieval process.

**Solution:** You are a helpful travel assistant. Ensure to gather and provide comprehensive information by retrieving all relevant data from your knowledge base or available APIs. Use tools when necessary and veri...
---



#### 3. The core technical cause is the AI's failure to accurately interpret and prioritize user-defined budgetary and class preferences during decision-making processes.

**Evidence:** 2 conversations  
**Type:** prompt

**Problem:** This modification explicitly instructs the AI to prioritize user-defined budgetary and class preferences, which addresses the core issue identified in the systemic failure pattern. By emphasizing these priorities at the beginning of the prompt, the AI is more likely to consider them throughout its decision-making process. The addition maintains existing functionality by keeping the original directive to use tools when necessary.

**Solution:** You are a helpful travel assistant. Always prioritize interpreting and adhering to the user's budgetary and class preferences during decision-making. Use tools when necessary....
---



#### 4. The AI failed to apply search filters for budget and amenities due to either a logic error in query processing or incomplete user intent extraction.

**Evidence:** 1 conversations  
**Type:** prompt

**Problem:** The original prompt lacked explicit instructions for the AI to focus on extracting budget and amenities information, which are crucial for applying accurate search filters. By emphasizing the extraction of these key details, the AI is better guided to process user intents regarding budget and amenities, thereby addressing the logic error or intent extraction issue. This change remains localized and preserves the existing functionality, as it builds upon the current task without altering its core instruction.

**Solution:** You are a helpful travel assistant. When understanding user requests, ensure to extract key details such as budget constraints and desired amenities. Apply appropriate search filters based on these pa...
---



#### 5. The AI misinterprets relative date phrases like 'next week' as specific dates without verifying with the user, due to inadequate context handling.

**Evidence:** 1 conversations  
**Type:** prompt

**Problem:** By explicitly instructing the AI to clarify relative date phrases with the user, we address the systemic failure of misinterpreting these phrases as specific dates. This change ensures that the AI gains the necessary context to provide accurate information, thereby fixing the issue while maintaining all existing functionalities.

**Solution:** You are a helpful travel assistant. Use tools when necessary. Ensure that relative date phrases such as 'next week', 'tomorrow', etc., are clarified with the user to ensure accurate context before pro...
---


In [5]:
# Display Batch Analysis Summary
print("📊 BATCH ANALYSIS RESULTS SUMMARY")
print("=" * 45)

# Extract key metrics
summary = result_travel_v1_batch["summary"]
evaluations = result_travel_v1_batch["final_evaluations"]
proposals = result_travel_v1_batch["analysis"]

print(f"Conversations Processed: {summary['total_conversations']}")
print(f"Evaluations Performed: {summary['total_evaluations']}")
print(f"Issues Detected: {summary['total_issues']}")
print(f"Average Score: {summary['average_score']:.3f}")
print(f"Proposals Generated: {summary['proposals_count']}")

# Show evaluation distribution
scores = [e.aggregate_score for e in evaluations]
excellent = sum(1 for s in scores if s >= 0.9)
good = sum(1 for s in scores if 0.8 <= s < 0.9)
fair = sum(1 for s in scores if 0.7 <= s < 0.8)
poor = sum(1 for s in scores if s < 0.7)

print(f"\nScore Distribution:")
print(f"  Excellent (≥0.9): {excellent}")
print(f"  Good (0.8-0.89): {good}")
print(f"  Fair (0.7-0.79): {fair}")
print(f"  Poor (<0.7): {poor}")

# Show top proposals
if proposals:
    print(f"\nTop {min(3, len(proposals))} Proposals:")
    for i, prop in enumerate(proposals[:3], 1):
        evidence_count = len(prop.evidence_ids)
        print(f"  {i}. {prop.failure_pattern} ({evidence_count} conversations)")

print(f"\n✅ Batch analysis complete - {len(proposals)} improvement proposals generated!")
print("🔍 Now demonstrating LLM judge depth on individual conversations...")

📊 BATCH ANALYSIS RESULTS SUMMARY
Conversations Processed: 15
Evaluations Performed: 15
Issues Detected: 20
Average Score: 0.723
Proposals Generated: 5

Score Distribution:
  Excellent (≥0.9): 5
  Good (0.8-0.89): 0
  Fair (0.7-0.79): 2
  Poor (<0.7): 8

Top 3 Proposals:
  1. The core technical cause is the assistant's inability to handle missing or incorrectly formatted input parameters and its failure to guide users through error recovery. These interactions lack proactive clarification and fail to solicit necessary user information for successful task completion. (15 conversations)
  2. The AI system fails to retrieve or communicate all necessary data from its knowledge base or APIs, leading to partial responses. (1 conversations)
  3. The core technical cause is the AI's failure to accurately interpret and prioritize user-defined budgetary and class preferences during decision-making processes. (2 conversations)

✅ Batch analysis complete - 5 improvement proposals generated!
🔍 Now d

From the batch analysis results above, we identified `travel_v1_issue_010` as a **perfect candidate** for demonstrating LLM judge capabilities:

**Heuristic Evaluation Results:**
- ✅ **Score: 1.00** (technically perfect)
- ✅ **Issues: 0** (passed all checks)
- ✅ **Evaluators: 3** (only heuristic stage - no LLM needed)

**What This Means:**
- The conversation passed ALL technical validation checks
- Tool calls were properly formatted and valid
- No obvious format, latency, or parameter issues detected
- **System deemed it "good enough" and skipped expensive LLM analysis**

### What LLM Judge Should Reveal

Despite passing heuristics, this conversation has **hidden semantic issues** that only deep language understanding can detect:

**User Request:** "I need flights from New York to Paris, I want business class with meal service included, departing in the morning, budget up to $1500."

**Assistant Response:** Made valid tool call to `flight_search` with only `origin` and `destination` parameters.

**Hidden Issues (Expected LLM Detection):**
- ❌ **Business class preference ignored** - No `class` parameter used
- ❌ **Morning departure ignored** - No time preferences considered
- ❌ **Meal service ignored** - No amenity filtering
- ❌ **Budget constraint ignored** - No price filtering
- ❌ **Generic response** - Just "Found 3 flights" without details

### Expected Contrast

**Heuristic Evaluation:** ✅ "Technically sound API call"
**LLM Judge Evaluation:** ❌ "Misses user intent and provides inadequate service"

This demonstrates why **staged evaluation** is crucial - technical validation alone is insufficient for quality AI agent assessment.

In [17]:
from src.db.repository import get_repository
from src.evaluation.service import EvaluationService
from src.evaluation.evaluators import get_global_registry, EvaluatorDiscovery

# Initialize repository (THIS WAS MISSING!)
repository = get_repository(data_dir="./data")

# Create LLM-only evaluator
registry = get_global_registry()
EvaluatorDiscovery.discover_and_register(registry)
llm_service = EvaluationService(repository, registry, enabled_evaluators=["llm_judge"])

# Evaluate conversation that passed heuristics
result = llm_service.evaluate("travel_v1_issue_010")
print(f"LLM Score: {result.aggregate_score}")
print(f"Issues Found: {len(result.issues)}")
for issue in result.issues:
    print(f"- {issue.issue_type.value}: {issue.description}")

LLM Score: 0.5666666666666668
Issues Found: 2
- low_helpfulness: The assistant did not account for the user's preferences for business class, meal service, morning departure, and budget limit.
- low_quality: The assistant provided a generic response without addressing the specific needs of the user.


✅